In [1]:
import json
import sys
sys.path.append('../..')
import pandas as pd
import ta
from utilities.data_manager import ExchangeDataManager
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
class Strategy():
    def __init__(
        self,
        pair,
        type=["long"],
        params={},
    ):
        self.df_pair = None
        self.df = None
        self.pair = pair
        self.initial_wallet = 1000
        self.use_long = "long" in type
        self.use_short = "short" in type
        self.params = params
        self.result_df = None

    def get_pair_data(self, timeframe, start = 2050, end = 2050):
        exchange = ExchangeDataManager(
            exchange_name=exchange_name,
            path_download="./database/exchanges"
        )

        self.df_pair = exchange.load_data(self.pair, timeframe, start, end)

    def populate_indicators2(self):
        params = self.params
        df = self.df_pair.copy()
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']),
            inplace=True
        )
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
        df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
        df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
        df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)
        #df['close_long_signal'] = df["rsi"].shift(1) >= 70

        df_signal = df.loc[
            df["open_long_signal"] | df["close_long_signal"],
            ["open_long_signal", "close_long_signal","open", "close"]
        ]
        df_signal["open_signal_lag"] = df_signal["open_long_signal"].shift(fill_value=False)
        df_signal["close_signal_lag"] = df_signal["close_long_signal"].shift(fill_value=False)
        df_first_signal  = df_signal[
            (~ df_signal["open_signal_lag"] & (df_signal["open_long_signal"] | df_signal["open_long_signal"].isnull())) |
            (~ df_signal["close_signal_lag"] & df_signal["close_long_signal"])
         ]
        df_first_signal["open_signal_lag"] = df_first_signal["open_long_signal"].shift(fill_value=False)
        df_first_signal["close_signal_lag"] = df_first_signal["close_long_signal"].shift(fill_value=False)

        df_order_tmp = df_first_signal[
            (df_first_signal["open_long_signal"] & (~df_first_signal["open_signal_lag"] | df_first_signal["open_long_signal"].isnull())) |
            (df_first_signal["close_long_signal"] & ~ df_first_signal["close_signal_lag"])
        ]
        df_order = df_order_tmp.loc[
            ~ ( ~ df_order_tmp["close_signal_lag"] & ~ df_order_tmp["open_signal_lag"] & df_order_tmp["close_long_signal"])
        ]
        df_order["order_number"] = df_order["open_long_signal"].cumsum()
        df_order["open_lag"] = df_order["open"].shift(-1)
        df_order["open_order"] = df_order["open"].shift()
        df_order.loc[df_order["open_long_signal"], "open_order"] = df_order.loc[df_order["open_long_signal"], "open"]
        df_pair = df[["open", "close", "low", "high", "mrat", "rsi", "mean_mrat", "stdev_mrat", "open_long_signal", "close_long_signal"]]

        leverage = params["leverage"]  # Fixed leverage
        maintenance_margin_percent = 0.004
        wallet = 1000  # Initial wallet balance
        quantity = 0  # Initial quantity

        # Ensure the DataFrame has 'quantity' and 'trade_result' columns initialized
        df_order['quantity'] = 0.0
        df_order['trade_result'] = 0.0
        df_order['trade_result_pct'] = 0.0

        # Iterating over DataFrame rows to process trading signals
        for i, row in df_order.iterrows():
            # Check if there is a signal to open a long position
            if row['open_long_signal']:
                # Calculate the new quantity based on the current wallet and leverage
                quantity = wallet * leverage / row['open']
                # Update the 'quantity' column with the new quantity
                df_order.at[i, 'quantity'] = quantity
                # No change in wallet yet as the position has just opened
                df_order.at[i, 'wallet'] = wallet
                # Track the price at which the position was opened
                open = row['open']
            elif row['close_long_signal']:
                # Calculate the trade result based on the difference between current and open price
                trade_result = (row['open'] - open) * quantity
                # Update the 'trade_result' column with the result of the closed trade
                df_order.at[i, 'trade_result'] = trade_result
                df_order.at[i, 'trade_result_pct'] = trade_result / wallet * 100
                # Update the wallet with the result of the trade
                wallet += trade_result
                # Reset quantity as the trade is closed
                df_order.at[i, 'quantity'] = quantity
                quantity = 0

            # Update the wallet and quantity for the current row
            df_order.at[i, 'wallet'] = wallet

        df_order_tmp = df_order[
            ["order_number", "quantity", "trade_result", "trade_result_pct", "wallet", "open_order"]
        ]
        df_order_final = df_pair.join(df_order_tmp)

        f = df_order_final['order_number'].ffill()
        b = df_order_final['order_number'].bfill()

        df_order_final['order_number'] = df_order_final['order_number'].mask(f == b, f)

        f = df_order_final['open_order'].ffill()
        b = df_order_final['open_order'].bfill()

        df_order_final['open_order'] = df_order_final['open_order'].mask(f == b, f)
        df_order_final['wallet'] = df_order_final['wallet'].ffill()
        #df_order_final["hypothetical_wallet"] = df_order_final["wallet"] + df_order_final["quantity"] * (df_order_final['open'] - df_order_final["open_order"])
        df_order_final["hypothetical_wallet"] = df_order_final["wallet"].shift() + df_order_final["quantity"] * (df_order_final['open'] - df_order_final["open_order"])
        df_order_final["hypothetical_low_result"] =  ((df_order_final["quantity"] * df_order_final["low"]) - df_order_final["wallet"]) / df_order_final["wallet"]
        df_order_final["drawdown"] = (df_order_final["low"] - df_order_final["open_order"]) / df_order_final["open_order"] * 100 * leverage
        df_order_final["is_liquidated"] = df_order_final['hypothetical_wallet'] < (df_order_final["wallet"] / leverage) * maintenance_margin_percent

        self.df = df_order_final

    def get_result_df(self):
        try:
            df = self.df
            if df is not None:
                total_trades = df.order_number.max()
                final_wallet_amount = df.loc[df["open_long_signal"], "wallet"].tail(1)
                total_profit = final_wallet_amount - self.initial_wallet
                total_profit_perc = total_profit / self.initial_wallet * 100
                avg_trade_profit_perc = df["trade_result_pct"].dropna().mean()
                avg_trade_profit = df["trade_result"].dropna().mean()
                max_drawdown = df["drawdown"].min()

                result_df = pd.DataFrame(
                    {
                        "params": str(self.params),
                        "final_wallet_amount": final_wallet_amount,
                        "total_profit": total_profit,
                        "total_profit_perc": total_profit_perc,
                        "total_trades": total_trades,
                        "avg_trade_profit_perc": avg_trade_profit_perc,
                        "avg_trade_profit": avg_trade_profit,
                        "max_drawdown": max_drawdown,
                    }
                )

                return result_df
            else:
                return None
        except Exception as e:
            print(e)
            print(self.params)
            return None

In [157]:
params = {
    "fast_ma": 5,
    "slow_ma": 60,
    "sigma_open": 2.5,
    "sigma_close": 2.7,
    "mean_mrat_lenght": 60,
    "leverage": 1
}

pair = "API3/USDT:USDT"
exchange_name = "binance"
tf = '15m'
start_date = "2023-01-01 00:00:00"
end_date = "2024-12-31 00:00:00"

In [162]:
strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date, end=end_date)
df2 = strat.populate_indicators2()
result = strat.get_result_df()

In [159]:
result

,params,final_wallet_amount,total_profit,total_profit_perc,total_trades,avg_trade_profit_perc,avg_trade_profit,max_drawdown
date,,,,,,,,
2024-03-18 02:00:00,"{'fast_ma': 5, 'slow_ma': 60, 'sigma_open': 2....",5724.194948,4724.194948,472.419495,129.0,0.722126,17.928094,-22.495274


In [174]:
df = strat.df
sum(df["trade_result"] > 0) / df["order_number"].max()

0.7251908396946565

# Check best params result

## API3

# TradingView consistence check

In [60]:
token = "API3"
pair = f"{token}/USDT:USDT"
exchange_name = "mexc"
tf = '15m'

In [61]:
df_result = pd.read_csv(f"{token}_mexc_15m_2022-01-01_2023-12-31_2.csv")

In [62]:
df_result["strat_position"] = df_result['final_wallet_amount'].rank(method='min', ascending=False)

In [63]:
df_result.sort_values("final_wallet_amount", ascending=False).head(100)

,date,params,final_wallet_amount,total_profit,total_profit_perc,total_trades,avg_trade_profit_perc,avg_trade_profit,max_drawdown,hold_profit,winrate,start_date,strat_position
4000,2023-12-28 16:00:00,"{'fast_ma': 7, 'slow_ma': 50, 'sigma_open': 2....",6082.327385,5082.327385,508.232739,248.0,0.427288,10.166317,-26.345328,-65.038373,0.608871,2022-02-22 03:45:00,1.0
4001,2023-12-28 16:00:00,"{'fast_ma': 7, 'slow_ma': 50, 'sigma_open': 2....",6066.264133,5066.264133,506.626413,236.0,0.453560,10.649439,-30.390016,-65.038373,0.588983,2022-02-22 03:45:00,2.0
2000,2023-12-28 16:00:00,"{'fast_ma': 6, 'slow_ma': 50, 'sigma_open': 2....",5883.533220,4883.533220,488.353322,253.0,0.411842,9.575101,-26.345328,-65.038373,0.604743,2022-02-22 03:45:00,3.0
28131,2023-12-26 23:45:00,"{'fast_ma': 35, 'slow_ma': 55, 'sigma_open': 2...",5843.518788,4843.518788,484.351879,132.0,0.776983,18.167869,-22.691976,-65.038373,0.606061,2022-02-22 03:45:00,4.0
2285,2023-12-28 16:00:00,"{'fast_ma': 6, 'slow_ma': 55, 'sigma_open': 2....",5063.103116,4063.103116,406.310312,198.0,0.558820,10.302989,-35.585971,-65.038373,0.636364,2022-02-22 03:45:00,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2023-12-28 15:45:00,"{'fast_ma': 5, 'slow_ma': 55, 'sigma_open': 2....",3269.596845,2269.596845,226.959685,154.0,0.504868,7.320679,-31.669219,-65.038373,0.655844,2022-02-22 03:45:00,96.0
4120,2023-12-30 04:00:00,"{'fast_ma': 7, 'slow_ma': 50, 'sigma_open': 2....",3265.622406,2265.622406,226.562241,328.0,0.248939,3.458966,-33.539276,-65.038373,0.600610,2022-02-22 03:45:00,97.0
4471,2023-12-28 16:30:00,"{'fast_ma': 7, 'slow_ma': 80, 'sigma_open': 2....",3255.161341,2255.161341,225.516134,153.0,0.477926,7.297874,-31.487696,-65.038373,0.666667,2022-02-22 03:45:00,98.0
386,2023-12-28 16:00:00,"{'fast_ma': 5, 'slow_ma': 60, 'sigma_open': 2....",3253.090920,2253.090920,225.309092,177.0,0.531920,6.395302,-49.359239,-65.038373,0.638418,2022-02-22 03:45:00,99.0


In [42]:
params = {
    "fast_ma": 8,
    "slow_ma": 50,
    "sigma_open": 2.6,
    "sigma_close": 2.5,
    "mean_mrat_lenght": 50,
    "leverage": 2
}

pair = "DOT/USDT:USDT"
exchange_name = "binance"
tf = '15m'
start_date = "2024-01-01 00:00:00"
end_date = "2024-03-14 01:00:00"

strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date, end=end_date)
strat.populate_indicators2()
df_result_tmp = strat.get_result_df()
df = strat.df

In [43]:
df.loc[df["order_number"] == 1].iloc[[0,-1],:]

,open,close,low,high,mrat,rsi,mean_mrat,stdev_mrat,open_long_signal,close_long_signal,order_number,quantity,trade_result,trade_result_pct,wallet,open_order,hypothetical_wallet,hypothetical_low_result,drawdown,is_liquidated
date,,,,,,,,,,,,,,,,,,,,
2024-01-02 15:45:00,8.513,8.425,8.406,8.539,0.983964,31.362024,1.004613,0.006958,True,False,1.0,234.934806,0.000000,0.000000,1000.000000,8.513,NaN,0.974862,-2.513802,False
2024-01-04 15:00:00,7.844,7.827,7.796,7.887,1.009856,70.046742,0.999612,0.005440,False,True,1.0,234.934806,-157.171385,-15.717138,842.828615,8.513,842.828615,1.173101,-16.844826,False


In [44]:
df.loc[df["order_number"] == 1]

,open,close,low,high,mrat,rsi,mean_mrat,stdev_mrat,open_long_signal,close_long_signal,order_number,quantity,trade_result,trade_result_pct,wallet,open_order,hypothetical_wallet,hypothetical_low_result,drawdown,is_liquidated
date,,,,,,,,,,,,,,,,,,,,
2024-01-02 15:45:00,8.513,8.425,8.406,8.539,0.983964,31.362024,1.004613,0.006958,True,False,1.0,234.934806,0.000000,0.000000,1000.000000,8.513,NaN,0.974862,-2.513802,False
2024-01-02 16:00:00,8.425,8.398,8.371,8.469,0.980113,29.858230,1.003917,0.007604,True,False,1.0,NaN,NaN,NaN,1000.000000,8.513,NaN,NaN,-3.336074,False
2024-01-02 16:15:00,8.397,8.456,8.383,8.461,0.977826,36.861886,1.003188,0.008292,True,False,1.0,NaN,NaN,NaN,1000.000000,8.513,NaN,NaN,-3.054152,False
2024-01-02 16:30:00,8.456,8.479,8.437,8.484,0.977051,39.444082,1.002452,0.008922,True,False,1.0,NaN,NaN,NaN,1000.000000,8.513,NaN,NaN,-1.785505,False
2024-01-02 16:45:00,8.479,8.498,8.458,8.525,0.977425,41.569988,1.001737,0.009451,True,False,1.0,NaN,NaN,NaN,1000.000000,8.513,NaN,NaN,-1.292141,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-04 14:00:00,7.700,7.701,7.669,7.710,1.006307,57.746273,0.998216,0.005410,False,False,1.0,NaN,NaN,NaN,1000.000000,8.513,NaN,NaN,-19.828498,False
2024-01-04 14:15:00,7.701,7.702,7.687,7.725,1.006229,57.906363,0.998493,0.005459,False,False,1.0,NaN,NaN,NaN,1000.000000,8.513,NaN,NaN,-19.405615,False
2024-01-04 14:30:00,7.702,7.778,7.692,7.784,1.006709,67.869850,0.998816,0.005457,False,False,1.0,NaN,NaN,NaN,1000.000000,8.513,NaN,NaN,-19.288148,False


In [45]:
df.loc[df["order_number"] == df["order_number"].max() - 1].iloc[[0,-1],:]

,open,close,low,high,mrat,rsi,mean_mrat,stdev_mrat,open_long_signal,close_long_signal,order_number,quantity,trade_result,trade_result_pct,wallet,open_order,hypothetical_wallet,hypothetical_low_result,drawdown,is_liquidated
date,,,,,,,,,,,,,,,,,,,,
2024-03-12 00:45:00,10.971,10.883,10.880,10.974,1.002110,32.816583,1.019511,0.005554,True,False,20.0,197.161225,0.000000,0.000000,1081.527898,10.971,1081.527898,0.983411,-1.658919,False
2024-03-13 07:15:00,11.060,11.033,11.009,11.093,1.016952,67.283100,1.009081,0.006704,False,True,20.0,197.161225,17.547349,1.622459,1099.075247,10.971,1099.075247,0.974886,0.692735,False


In [46]:
df_result_tmp

,params,final_wallet_amount,total_profit,total_profit_perc,total_trades,avg_trade_profit_perc,avg_trade_profit,max_drawdown
date,,,,,,,,
2024-03-13 15:30:00,"{'fast_ma': 8, 'slow_ma': 50, 'sigma_open': 2....",1099.075247,99.075247,9.907525,21.0,0.778221,5.183636,-49.195348


✅ Same first order as TV
✅ Same trade result
✅ Same profit

## Best params 2024 performance

In [64]:
top_params = df_result.sort_values("final_wallet_amount", ascending=False).iloc[:50]["params"]

In [68]:
train_start_date = "2023-01-01 00:00:00"
train_end_date = "2023-12-31 00:00:00"
test_start_date = "2024-01-01 00:00:00"
test_end_date = "2024-03-27 00:00:00"

result = []
for param in top_params:
    param = param.replace("'", "\"")
    param = json.loads(param)
    param["leverage"] = 1
    strat = Strategy(pair=pair, params=param)
    strat.get_pair_data(timeframe=tf, start=test_start_date, end=test_end_date)
    strat.populate_indicators2()
    df_result_tmp = strat.get_result_df()
    result.append(df_result_tmp)

In [69]:
result_dfs = pd.concat(result)

In [70]:
result_dfs

,params,final_wallet_amount,total_profit,total_profit_perc,total_trades,avg_trade_profit_perc,avg_trade_profit,max_drawdown
date,,,,,,,,
2024-03-26 15:45:00,"{'fast_ma': 7, 'slow_ma': 50, 'sigma_open': 2....",2042.169301,1042.169301,104.216930,26.0,1.457632,20.434692,-18.754177
2024-03-26 15:45:00,"{'fast_ma': 7, 'slow_ma': 50, 'sigma_open': 2....",2042.169301,1042.169301,104.216930,26.0,1.457632,20.434692,-18.754177
2024-03-26 15:45:00,"{'fast_ma': 6, 'slow_ma': 50, 'sigma_open': 2....",1820.736287,820.736287,82.073629,25.0,1.279110,16.749720,-19.626564
2024-03-24 04:15:00,"{'fast_ma': 35, 'slow_ma': 55, 'sigma_open': 2...",1348.197955,348.197955,34.819796,20.0,0.849842,9.110224,-18.738278
2024-03-26 16:15:00,"{'fast_ma': 6, 'slow_ma': 55, 'sigma_open': 2....",1847.145271,847.145271,84.714527,28.0,1.170699,15.402641,-19.626564
2024-03-26 15:45:00,"{'fast_ma': 6, 'slow_ma': 50, 'sigma_open': 2....",1820.736287,820.736287,82.073629,25.0,1.279110,16.749720,-19.626564
2024-03-26 15:45:00,"{'fast_ma': 6, 'slow_ma': 50, 'sigma_open': 2....",1820.736287,820.736287,82.073629,25.0,1.279110,16.749720,-19.626564
2024-03-26 16:00:00,"{'fast_ma': 7, 'slow_ma': 50, 'sigma_open': 2....",2041.844249,1041.844249,104.184425,27.0,1.403805,19.657439,-19.626564
2024-03-24 03:45:00,"{'fast_ma': 35, 'slow_ma': 55, 'sigma_open': 2...",1208.263201,208.263201,20.826320,17.0,0.675660,6.697423,-18.940983


In [71]:
result_dfs["strat_position"] = result_dfs['final_wallet_amount'].rank(method='min', ascending=False)

In [72]:
df = result_dfs.merge(
    df_result,
    how="left",
    on="params",
    suffixes=("_test", "_train")
).sort_values("final_wallet_amount_test", ascending=False)
df.reindex(sorted(df.columns), axis=1)

,avg_trade_profit_perc_test,avg_trade_profit_perc_train,avg_trade_profit_test,avg_trade_profit_train,date,final_wallet_amount_test,final_wallet_amount_train,hold_profit,max_drawdown_test,max_drawdown_train,...,start_date,strat_position_test,strat_position_train,total_profit_perc_test,total_profit_perc_train,total_profit_test,total_profit_train,total_trades_test,total_trades_train,winrate
45,1.757752,0.661157,27.134725,9.073246,2023-12-28 16:00:00,2438.140447,3727.691861,-65.038373,-19.626564,-48.766207,...,2022-02-22 03:45:00,1.0,46.0,143.814045,272.769186,1438.140447,2727.691861,27.0,151.0,0.649007
44,1.476277,0.641567,22.219881,8.646995,2023-12-28 16:00:00,2310.972995,3754.520684,-65.038373,-19.626564,-49.359239,...,2022-02-22 03:45:00,2.0,45.0,131.097299,275.452068,1310.972995,2754.520684,30.0,160.0,0.656250
10,1.277617,0.671468,18.823651,10.427030,2023-12-30 03:45:00,2223.537285,4472.200847,-65.038373,-19.626564,-49.359239,...,2022-02-22 03:45:00,3.0,11.0,122.353729,347.220085,1223.537285,3472.200847,33.0,167.0,0.658683
43,1.277617,0.550853,18.823651,7.486851,2023-12-30 03:45:00,2223.537285,3762.648061,-65.038373,-19.626564,-49.359239,...,2022-02-22 03:45:00,3.0,44.0,122.353729,276.264806,1223.537285,2762.648061,33.0,185.0,0.637838
41,1.279526,0.562180,18.590849,7.626423,2023-12-30 03:45:00,2171.223468,3768.391535,-65.038373,-19.626564,-49.359239,...,2022-02-22 03:45:00,5.0,42.0,117.122347,276.839154,1171.223468,2768.391535,32.0,182.0,0.648352
15,1.451759,0.369680,20.647481,6.712116,2023-12-28 16:15:00,2094.316519,4392.437424,-65.038373,-18.754177,-30.390016,...,2022-02-22 03:45:00,6.0,16.0,109.431652,339.243742,1094.316519,3392.437424,27.0,251.0,0.585657
27,1.129402,0.574033,15.930658,7.944064,2023-12-30 04:00:00,2067.354113,3947.247675,-65.038373,-20.920685,-49.359239,...,2022-02-22 03:45:00,7.0,28.0,106.735411,294.724767,1067.354113,2947.247675,34.0,186.0,0.639785
34,1.473560,0.363756,20.749753,6.201527,2023-12-28 16:00:00,2058.237392,3860.474849,-65.038373,-18.754177,-30.390016,...,2022-02-22 03:45:00,8.0,35.0,105.823739,286.047485,1058.237392,2860.474849,26.0,229.0,0.593886
1,1.457632,0.453560,20.434692,10.649439,2023-12-28 16:00:00,2042.169301,6066.264133,-65.038373,-18.754177,-30.390016,...,2022-02-22 03:45:00,9.0,2.0,104.216930,506.626413,1042.169301,5066.264133,26.0,236.0,0.588983
0,1.457632,0.427288,20.434692,10.166317,2023-12-28 16:00:00,2042.169301,6082.327385,-65.038373,-18.754177,-26.345328,...,2022-02-22 03:45:00,9.0,1.0,104.216930,508.232739,1042.169301,5082.327385,26.0,248.0,0.608871


In [52]:
df

,params,final_wallet_amount_test,total_profit_test,total_profit_perc_test,total_trades_test,avg_trade_profit_perc_test,avg_trade_profit_test,max_drawdown_test,strat_position_test,date,final_wallet_amount_train,total_profit_train,total_profit_perc_train,total_trades_train,avg_trade_profit_perc_train,avg_trade_profit_train,max_drawdown_train,hold_profit,winrate,strat_position_train
29,"{'fast_ma': 5, 'slow_ma': 115, 'sigma_open': 2...",2134.331350,1134.331350,113.433135,28.0,1.444409,20.624206,-19.399757,1.0,2023-12-28 18:00:00,5845.459421,4845.459421,484.545942,166.0,0.671448,14.638850,-45.687303,-47.060827,0.596386,30.0
31,"{'fast_ma': 5, 'slow_ma': 115, 'sigma_open': 2...",2134.331350,1134.331350,113.433135,28.0,1.444409,20.624206,-19.399757,1.0,2023-12-28 18:00:00,5823.573581,4823.573581,482.357358,132.0,0.886698,18.340584,-47.523557,-47.060827,0.613636,32.0
16,"{'fast_ma': 5, 'slow_ma': 115, 'sigma_open': 2...",2134.331350,1134.331350,113.433135,28.0,1.444409,20.624206,-19.399757,1.0,2023-12-28 18:00:00,6194.004810,5194.004810,519.400481,153.0,0.760908,17.029524,-45.687303,-47.060827,0.620915,17.0
35,"{'fast_ma': 6, 'slow_ma': 115, 'sigma_open': 2...",2052.640436,1052.640436,105.264044,28.0,1.372929,19.138917,-19.399757,4.0,2023-12-28 18:00:00,5766.265936,4766.265936,476.626594,159.0,0.696510,15.035539,-45.322727,-47.060827,0.622642,36.0
26,"{'fast_ma': 10, 'slow_ma': 115, 'sigma_open': ...",1906.399342,906.399342,90.639934,28.0,1.231207,16.479988,-21.012189,5.0,2023-12-28 18:45:00,5948.546598,4948.546598,494.854660,142.0,0.807666,17.486030,-46.218612,-47.060827,0.598592,27.0
13,"{'fast_ma': 10, 'slow_ma': 115, 'sigma_open': ...",1906.399342,906.399342,90.639934,28.0,1.231207,16.479988,-21.012189,5.0,2023-12-28 18:45:00,6273.421286,5273.421286,527.342129,131.0,0.900020,20.204679,-46.218612,-47.060827,0.587786,14.0
30,"{'fast_ma': 6, 'slow_ma': 120, 'sigma_open': 2...",1890.236126,890.236126,89.023613,27.0,1.271848,16.796908,-20.623086,7.0,2023-12-28 18:15:00,5832.496947,4832.496947,483.249695,144.0,0.772576,16.837968,-44.973515,-47.060827,0.625000,31.0
19,"{'fast_ma': 6, 'slow_ma': 120, 'sigma_open': 2...",1890.236126,890.236126,89.023613,27.0,1.271848,16.796908,-20.623086,7.0,2023-12-28 18:15:00,6070.284430,5070.284430,507.028443,159.0,0.707985,15.994588,-44.973515,-47.060827,0.635220,20.0
8,"{'fast_ma': 7, 'slow_ma': 120, 'sigma_open': 2...",1852.493747,852.493747,85.249375,27.0,1.232656,16.084788,-20.623086,9.0,2023-12-28 18:30:00,6460.137508,5460.137508,546.013751,153.0,0.758330,17.902090,-44.973515,-47.060827,0.627451,9.0
14,"{'fast_ma': 9, 'slow_ma': 120, 'sigma_open': 2...",1850.419262,850.419262,85.041926,26.0,1.270069,16.674887,-21.116698,10.0,2023-12-28 19:00:00,6222.262274,5222.262274,522.226227,158.0,0.721379,16.578610,-45.242063,-47.060827,0.613924,15.0
